In [18]:
import db_builder.db_handler as dbh
import numpy as np
from torchvision import transforms, datasets
from tqdm import tqdm
import logging
from getpass import getpass
from PIL import Image
import torchvision.models as models
import torch.nn as nn
import torch
import os
import cv2
from collections import Counter
from os import path

In [19]:
logging.getLogger('sqlalchemy.engine').propagate = False

db_params = {
    'user': 'postgres',
    'password': getpass('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'ttdatabase',  #tensionTerminator
}
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
engine = toolcheck.get_engine()

Connected to PostgreSQL, DB: ttdatabase


In [20]:
unlabeled_loops = toolcheck.get_unlabeled_loop_id()

2023-11-29 20:11:12,807 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-29 20:11:12,808 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-29 20:11:12,810 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-29 20:11:12,811 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-29 20:11:12,812 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-29 20:11:12,812 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-29 20:11:12,814 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-29 20:11:12,815 INFO sqlalchemy.engine.Engine SELECT id FROM loop WHERE human_labeled = false
2023-11-29 20:11:12,815 INFO sqlalchemy.engine.Engine [generated in 0.00070s] {}


In [21]:
unlabeled_loops

[129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150]

In [22]:
transforms_person = transforms.Compose([
    transforms.Resize(150),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [23]:
def dataset_path(model_folder: str,timestamp: str):
    abs_path = os.getcwd()
    two_up =  path.abspath(path.join(abs_path ,"../.."))
    return path.join(two_up, 'data', model_folder, timestamp)

In [24]:
person_dataset = datasets.ImageFolder(
    root=dataset_path('person_test', '29_11_2023_07_24_46'),
    transform=transforms_person
)

In [25]:
person_dataset.classes

['Christina_Greiderer',
 'Christine_Lackinger',
 'Juergen_Zangerl',
 'Lukas_Prenner',
 'MartinPO_Feuerstein',
 'Martin_Hofer',
 'Philipp_Egger',
 'Pirmin_Aster',
 'Robert_Goller',
 'Suganthi_Manoharan']

In [26]:
transfer_person_model = models.resnet152()
transfer_person_model.fc = nn.Sequential(
    nn.Linear(transfer_person_model.fc.in_features, 2048),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1024, 500),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(500, 10)
)
transfer_person_model_state_dict = torch.load("C:\\Users\\Pirmin.000\\PycharmProjects\\IGP\\models\\person_test\\29_11_2023_07_24_46\\model_29_11_2023_17_31_06.pt")
transfer_person_model.load_state_dict(transfer_person_model_state_dict)
transfer_person_model.to("cuda")
transfer_person_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [27]:
def checkImage(path: str, transfer_model, orig_set, transforms_wt):
    img = Image.open(path)
    img_tensor = transforms_wt(img).unsqueeze(0)
    prediction = transfer_model(img_tensor.to("cuda"))
    predicted_probabilities = torch.softmax(prediction, dim=1)
    predicted_class_idx = torch.argmax(prediction).item()
    predicted_class = orig_set.classes[predicted_class_idx]

    # Get the confidence score for the predicted class
    confidence = predicted_probabilities[0, predicted_class_idx].item() * 100  # Convert to percentage

    return predicted_class, confidence

In [28]:
def trigger_crop(image):
    crop_box = (400, 450, 550, 550)
    cropped_image = transforms.functional.crop(image, *crop_box)
    return cropped_image

In [29]:
def video_to_image_converter(source_path: str, output_path: str, crop=False):
    vidcap = cv2.VideoCapture(source_path)
    os.makedirs(output_path, exist_ok=True)
    count = 0

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    #fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    fps = 15

    while count < total_frames:
        success, image = vidcap.read()

        if count % int(fps) == 0:
            if not image is None and not image.size == 0:
                transform_test = transforms.ToPILImage()
                image = transform_test(image)
                if crop:
                    image = trigger_crop(image)
                image = np.asarray(image)
                cv2.imwrite(f"{output_path}/{count}.png", image)  # save frame as PNG file

        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [31]:
directory = "tmp_person"
if not os.path.exists(directory):
    os.makedirs(directory)

for video_source_id in tqdm(unlabeled_loops):
    video_source = toolcheck.get_filepath_by_loop_id(video_source_id)
    video_to_image_converter(video_source, directory, True)
    dataset = []
    conf = []
    
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        predicted_class, confidence = checkImage(f, transfer_person_model, person_dataset, transforms_person)
        conf.append(confidence)
        dataset.append(predicted_class)
    
        os.remove(f)
    os.rmdir(directory)
    dataset_confidence = sum(conf) / len(conf)
    print(Counter(dataset))
    user_name = Counter(dataset).most_common()[0][0].split('_')
    user_id = toolcheck.get_user_id_by_name(user_name[0],user_name[1])
    toolcheck.set_user_id_loops(user_id,video_source_id )

    print(f"Loop_id: {video_source_id}, Result:{user_name} ID:{user_id}")

  0%|          | 0/22 [00:00<?, ?it/s]

2023-11-29 20:19:17,066 INFO sqlalchemy.engine.Engine SELECT file_path FROM video WHERE loop_id = %(loop_id)s AND device = 'rgbCam'
2023-11-29 20:19:17,066 INFO sqlalchemy.engine.Engine [cached since 481.4s ago] {'loop_id': 129}
Counter({'Pirmin_Aster': 92, 'Martin_Hofer': 60, 'Robert_Goller': 2, 'Suganthi_Manoharan': 1})
2023-11-29 20:19:39,757 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.surename AS users_surename 
FROM users 
WHERE users.name = %(name_1)s AND users.surename = %(surename_1)s 
 LIMIT %(param_1)s
2023-11-29 20:19:39,757 INFO sqlalchemy.engine.Engine [cached since 482.5s ago] {'name_1': 'Pirmin', 'surename_1': 'Aster', 'param_1': 1}
2023-11-29 20:19:39,759 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop 
WHERE loop.id = %(id_1)

  5%|▍         | 1/22 [00:22<07:56, 22.70s/it]

Loop_id: 129, Result:['Pirmin', 'Aster'] ID:7
2023-11-29 20:19:39,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-29 20:19:39,765 INFO sqlalchemy.engine.Engine SELECT file_path FROM video WHERE loop_id = %(loop_id)s AND device = 'rgbCam'
2023-11-29 20:19:39,766 INFO sqlalchemy.engine.Engine [cached since 504.1s ago] {'loop_id': 130}
Counter({'Pirmin_Aster': 133, 'Martin_Hofer': 64, 'Juergen_Zangerl': 32, 'Suganthi_Manoharan': 9, 'Philipp_Egger': 2})
2023-11-29 20:20:15,915 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.surename AS users_surename 
FROM users 
WHERE users.name = %(name_1)s AND users.surename = %(surename_1)s 
 LIMIT %(param_1)s
2023-11-29 20:20:15,916 INFO sqlalchemy.engine.Engine [cached since 518.6s ago] {'name_1': 'Pirmin', 'surename_1': 'Aster', 'param_1': 1}
2023-11-29 20:20:15,918 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_lengt

  9%|▉         | 2/22 [00:58<10:12, 30.62s/it]

...

Loop_id: 148, Result:['Pirmin', 'Aster'] ID:7
2023-11-29 20:30:57,682 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-29 20:30:57,684 INFO sqlalchemy.engine.Engine SELECT file_path FROM video WHERE loop_id = %(loop_id)s AND device = 'rgbCam'
2023-11-29 20:30:57,684 INFO sqlalchemy.engine.Engine [cached since 1182s ago] {'loop_id': 149}
Counter({'Pirmin_Aster': 65, 'Martin_Hofer': 5})
2023-11-29 20:31:07,486 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.surename AS users_surename 
FROM users 
WHERE users.name = %(name_1)s AND users.surename = %(surename_1)s 
 LIMIT %(param_1)s
2023-11-29 20:31:07,486 INFO sqlalchemy.engine.Engine [cached since 1170s ago] {'name_1': 'Pirmin', 'surename_1': 'Aster', 'param_1': 1}
2023-11-29 20:31:07,488 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labe

 95%|█████████▌| 21/22 [11:50<00:22, 22.09s/it]

Loop_id: 149, Result:['Pirmin', 'Aster'] ID:7
2023-11-29 20:31:07,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-29 20:31:07,494 INFO sqlalchemy.engine.Engine SELECT file_path FROM video WHERE loop_id = %(loop_id)s AND device = 'rgbCam'
2023-11-29 20:31:07,495 INFO sqlalchemy.engine.Engine [cached since 1192s ago] {'loop_id': 150}
Counter({'Pirmin_Aster': 146, 'Juergen_Zangerl': 55, 'Martin_Hofer': 39, 'Suganthi_Manoharan': 3, 'Philipp_Egger': 1})
2023-11-29 20:31:40,249 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.surename AS users_surename 
FROM users 
WHERE users.name = %(name_1)s AND users.surename = %(surename_1)s 
 LIMIT %(param_1)s
2023-11-29 20:31:40,250 INFO sqlalchemy.engine.Engine [cached since 1203s ago] {'name_1': 'Pirmin', 'surename_1': 'Aster', 'param_1': 1}
2023-11-29 20:31:40,252 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length,

100%|██████████| 22/22 [12:23<00:00, 33.78s/it]

Loop_id: 150, Result:['Pirmin', 'Aster'] ID:7
